In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [7]:
input_text = au.read_txt_file_lines('input.txt')
n_rows = len(input_text)

list_module_names = []
for l in input_text:
    tmp = l.split() 
    assert tmp[0] not in list_module_names
    list_module_names.append(tmp[0])
print('All modules/definitions are unique')

All modules/definitions are unique


### More info:
- **pulse**: `0` (low) or `1` (high)
- **broadcast module**: when it receives a pulse, it sends the same pulse to all its destination modules 
- **button module**: when activated, it send a single low pulse to broadcast module 
- **conjunction module**: `&` prefix. 1) remembers most recent pusle received from EACH connected input module. 2) default to low pulse for each. 3) when it receives a pulse: a) update memory, b) if ALL memories are high, send low. Else, send high. 
- **flip flop**: `%` prefix. Default: off. if receives high pulse, pass. if receives low pulse, switch on/off. If new state is on; send high pulse. if new state is off; send low pulse. 
- **untyped**: no definition. nothing happens. 

- Process pulses in order they are sent. (deque?)



In [8]:
pulse = collections.namedtuple('pulse', 'source state dest') 

class Mod():
    def __init__(self, name, mod_type, list_dest):
        assert mod_type in ['flipflop', 'conjunction', 'broadcast', 'button', 'untyped'], f' {mod_type} is not a valid mod_type'
        self.name = name
        self.mod_type = mod_type
        self.list_dest = list_dest

        if mod_type == 'flipflop':
            self.flipflop_state = 0
        elif mod_type == 'conjunction':
            self.list_input_mods = []

    def activate(self, input_pulse):
        state_pulse = input_pulse.state
        assert input_pulse.dest == self.name

        if self.mod_type == 'broadcast':
            return [pulse(self.name, state_pulse, dest) for dest in self.list_dest]
        elif self.mod_type == 'conjunction':
            assert input_pulse.source in self.list_input_mods
            self.conjunction_memory[input_pulse.source] = state_pulse
            if np.all(list(self.conjunction_memory.values())):
                return [pulse(self.name, 0, dest) for dest in self.list_dest]
            else:
                return [pulse(self.name, 1, dest) for dest in self.list_dest]
        elif self.mod_type == 'flipflop':
            if state_pulse == 1:
                return None 
            else:
                self.flipflop_state = 1 - self.flipflop_state
                return [pulse(self.name, self.flipflop_state, dest) for dest in self.list_dest]
        elif self.mod_type == 'button':
            assert False, 'Button should not be activated'
        elif self.mod_type == 'untyped':   
            return None 
        else:
            assert False, 'Unknown mod_type'

    def create_conjunction_memory(self):
        assert self.mod_type == 'conjunction'
        assert len(self.list_input_mods) > 0
        self.conjunction_memory = {mod_name: 0 for mod_name in self.list_input_mods}
        
    def press_button(self):
        assert self.mod_type == 'button'
        return [pulse(self.name, 0, 'broadcaster')]
    
def get_state_all_mods(dict_mods):
    list_states = []
    for k, v in dict_mods.items():
        if v.mod_type == 'flipflop':
            list_states.append(v.flipflop_state)
        elif v.mod_type == 'conjunction':
            list_states.append(v.conjunction_memory)
        else:
            list_states.append(None)
    return tuple(list_states)


In [22]:
dict_mods = {}
for l in input_text:
    tmp = l.split(' ->')
    assert len(tmp) == 2
    source_name = tmp[0]
    assert source_name not in dict_mods
    list_dest = tmp[1].split(',')
    list_dest = [d.strip() for d in list_dest]

    if source_name.startswith('%'):
        mod_type = 'flipflop'
        source_name = source_name[1:]
    elif source_name.startswith('&'):
        mod_type = 'conjunction'
        source_name = source_name[1:]
    elif source_name == 'broadcaster':
        mod_type = 'broadcast'
    else:
        assert False, f'Unknown mod_type for {source_name}'

    dict_mods[source_name] = Mod(source_name, mod_type, list_dest)

# assert len(dict_mods) == 58
dict_mods['button'] = Mod('button', 'button', [])

## create conjunction memory
dict_mods_untyped = {}
list_input_to_qt = []
for mod_name, mod in dict_mods.items():
    if mod.mod_type == 'conjunction':
        for mod_name_tmp, mod_tmp in dict_mods.items():
            if mod_name in mod_tmp.list_dest:
                mod.list_input_mods.append(mod_name_tmp)

        mod.create_conjunction_memory()
    for dest_name in mod.list_dest:
        if dest_name not in dict_mods:
            dict_mods_untyped[dest_name] = Mod(dest_name, 'untyped', [])
        if dest_name == 'qt':
            list_input_to_qt.append(mod_name)

dict_mods.update(dict_mods_untyped)
print(f'Number of mods: {len(dict_mods)}')
print(f'Following input nodes to qt: {list_input_to_qt}')

Number of mods: 60
Following input nodes to qt: ['mr', 'kk', 'gl', 'bb']


In [23]:
pulses_deque = collections.deque() 

n_pulses_high, n_pulses_low = 0, 0
n_buttons = 100000
i_save= []
curr_total_state = get_state_all_mods(dict_mods)
list_all_states = []
dict_n_button_presses_to_reach_node = {x: [] for x in list_input_to_qt}
for i in tqdm(range(n_buttons)):
    list_all_states.append(curr_total_state)
    start_pulse = dict_mods['button'].press_button()
    assert len(start_pulse) == 1
    pulses_deque.extend(start_pulse)

    while len(pulses_deque) > 0:
        curr_pulse = pulses_deque.popleft()
        if curr_pulse.state == 1:
            n_pulses_high += 1
        elif curr_pulse.state == 0:
            n_pulses_low += 1
        # print(curr_pulse)
            

        if curr_pulse.source in list_input_to_qt and curr_pulse.state == 1:
            dict_n_button_presses_to_reach_node[curr_pulse.source].append(i + 1)

        # if curr_pulse.dest == 'rx':
        #     if curr_pulse.state == 0:
        #         i_save.append(i + 1)
        #         print('part 2: low rx', i + 1)

        curr_mod = dict_mods[curr_pulse.dest]
        new_pulses = curr_mod.activate(curr_pulse)
        if new_pulses is not None:
            pulses_deque.extend(new_pulses)

    new_total_state = get_state_all_mods(dict_mods)
    if new_total_state in list_all_states:
        print('part 2: stable after', i + 1)
        break
    
    curr_total_state = new_total_state

if n_buttons == 1000:
    print(f'part 1: {n_pulses_high * n_pulses_low}')

100%|██████████| 100000/100000 [01:16<00:00, 1307.21it/s]


## part 2:
attempt brute-forcing did not find a result, but also showed that the initial state was never repeated. After seeing a hint, I then looked for the cycles that each input node to `qt` activates and find first time they all activate:

In [34]:
for k , v in dict_n_button_presses_to_reach_node.items():
    print(k, v[:5], np.unique(np.diff(v)))
    print(dict_mods[k].mod_type)

list_intervals_input_nodes = [v[0] for k, v in dict_n_button_presses_to_reach_node.items()]

mr [3907, 7814, 11721, 15628, 19535] [3907]
conjunction
kk [3931, 7862, 11793, 15724, 19655] [3931]
conjunction
gl [3989, 7978, 11967, 15956, 19945] [3989]
conjunction
bb [3967, 7934, 11901, 15868, 19835] [3967]
conjunction


In [36]:
for ii in range(len(list_intervals_input_nodes)):
    for jj in range(ii + 1, len(list_intervals_input_nodes)):
        assert au.gcd(list_intervals_input_nodes[ii], list_intervals_input_nodes[jj]) == 1

np.prod(list_intervals_input_nodes)

243037165713371